# Introduction
(숙제 설명)

## Import

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.datasets import cifar10, fashion_mnist
import math 

seed = 777
np.random.seed(seed)
tf.random.set_random_seed(seed) # for reproducibility

## Helper Functions

In [2]:
%matplotlib inline
def plot_fn(labels, train_loss_histories, train_acc_histories, test_acc_histories):
    fig = plt.figure(figsize=(18,5))
    ax1 = fig.add_subplot(1, 3, 1)
    ax2 = fig.add_subplot(1, 3, 2)
    ax3 = fig.add_subplot(1, 3, 3)
    for label, train_loss_history, train_acc_history, test_acc_history in zip(labels, train_loss_histories, train_acc_histories, test_acc_histories):
        ax1.plot(train_loss_history, label=str(label))
        ax2.plot(train_acc_history, label=str(label))
        ax3.plot(test_acc_history, label=str(label))
        
    ax1.set_xlabel('Batch #')
    ax1.set_ylabel('Training Loss [entropy]')
    ax2.set_xlabel('Batch #')
    ax2.set_ylabel('Training Accuracy')
    ax3.set_xlabel('Batch #')
    ax3.set_ylabel('Test Accuracy')
    plt.legend()
    plt.show()

## Hyperparameters

In [3]:
# Hyper paramenters
batch_size = 1024
epochs = 200
aug_scale = 5

## Dataset 
(데이터 추후 결정)

In [4]:
from keras.preprocessing.image import ImageDataGenerator

# (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data() # cifar10.load_data()
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype(np.float32)[:1000] / 255. 
x_train += np.random.normal(0, 0.3, x_train.shape)
y_train = y_train.astype(np.int64)[:1000]
x_test = x_test.astype(np.float32) / 255.
# x_test += np.random.normal(0, 0.3, x_test.shape)
y_test = y_test.astype(np.int64)

generator = ImageDataGenerator(rotation_range=20,
                       width_shift_range=0.1,
                       height_shift_range=0.1)

data_flow = generator.flow(x_train, y_train,
                            batch_size=batch_size)

print("Train input shape: {}".format(x_train.shape))
print("Train label shape: {}".format(y_train.shape))
print("Test input shape: {}".format(x_test.shape))
print("Test label shape: {}".format(y_test.shape))

Using TensorFlow backend.


Train input shape: (1000, 32, 32, 3)
Train label shape: (1000, 1)
Test input shape: (10000, 32, 32, 3)
Test label shape: (10000, 1)


## Model

In [5]:
# model 정의
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout, BatchNormalization

def create_model(optimizer='sgd', init='glorot_uniform', drop_rate=0.0, batch_norm=False):
    model = keras.Sequential()
    # input layer
    model.add(Flatten(input_shape=(32, 32, 3)))
    # hidden layers
    model.add(Dense(2048,kernel_initializer=init))
    if batch_norm:
        model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(drop_rate))
    
    model.add(Dense(2048,kernel_initializer=init))
    if batch_norm:
        model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(drop_rate))
    
    # output layer
    model.add(Dense(10,kernel_initializer=init))
    model.add(Activation('softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
    return model

In [6]:
# model 생성
model = create_model()
model.summary()

W0718 20:42:10.620637 139676478900032 deprecation.py:506] From /home/yongsu/anaconda3/envs/tf114/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              6293504   
_________________________________________________________________
activation (Activation)      (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
activation_1 (Activation)    (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0

In [7]:
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))
# hist = model.fit_generator(data_flow, epochs=epochs, steps_per_epoch=math.ceil(len(x_train) / batch_size), validation_data=(x_test, y_test))

Train on 1000 samples, validate on 10000 samples
Epoch 1/200
1000/1000 [==============================] - 0s 392us/sample - loss: 2.4359 - acc: 0.0920 - val_loss: 2.3092 - val_acc: 0.1018
Epoch 2/200
1000/1000 [==============================] - 0s 142us/sample - loss: 2.3116 - acc: 0.1050 - val_loss: 2.2928 - val_acc: 0.1251
Epoch 3/200
1000/1000 [==============================] - 0s 137us/sample - loss: 2.2860 - acc: 0.1280 - val_loss: 2.2841 - val_acc: 0.1356
Epoch 4/200
1000/1000 [==============================] - 0s 159us/sample - loss: 2.2709 - acc: 0.1460 - val_loss: 2.2761 - val_acc: 0.1413
Epoch 5/200
1000/1000 [==============================] - 0s 144us/sample - loss: 2.2575 - acc: 0.1550 - val_loss: 2.2683 - val_acc: 0.1477
Epoch 6/200
1000/1000 [==============================] - 0s 135us/sample - loss: 2.2446 - acc: 0.1680 - val_loss: 2.2608 - val_acc: 0.1515
Epoch 7/200
1000/1000 [==============================] - 0s 138us/sample - loss: 2.2321 - acc: 0.1790 - val_loss: 2.2

Epoch 60/200
1000/1000 [==============================] - 0s 135us/sample - loss: 1.7997 - acc: 0.5150 - val_loss: 2.0618 - val_acc: 0.2618
Epoch 61/200
1000/1000 [==============================] - 0s 144us/sample - loss: 1.7937 - acc: 0.5190 - val_loss: 2.0600 - val_acc: 0.2627
Epoch 62/200
1000/1000 [==============================] - 0s 147us/sample - loss: 1.7877 - acc: 0.5210 - val_loss: 2.0581 - val_acc: 0.2628
Epoch 63/200
1000/1000 [==============================] - 0s 132us/sample - loss: 1.7817 - acc: 0.5250 - val_loss: 2.0563 - val_acc: 0.2634
Epoch 64/200
1000/1000 [==============================] - 0s 133us/sample - loss: 1.7758 - acc: 0.5280 - val_loss: 2.0545 - val_acc: 0.2644
Epoch 65/200
1000/1000 [==============================] - 0s 142us/sample - loss: 1.7698 - acc: 0.5310 - val_loss: 2.0528 - val_acc: 0.2651
Epoch 66/200
1000/1000 [==============================] - 0s 159us/sample - loss: 1.7640 - acc: 0.5370 - val_loss: 2.0511 - val_acc: 0.2662
Epoch 67/200
1000/10

Epoch 119/200
1000/1000 [==============================] - 0s 131us/sample - loss: 1.4908 - acc: 0.6830 - val_loss: 1.9892 - val_acc: 0.2963
Epoch 120/200
1000/1000 [==============================] - 0s 136us/sample - loss: 1.4861 - acc: 0.6860 - val_loss: 1.9884 - val_acc: 0.2965
Epoch 121/200
1000/1000 [==============================] - 0s 130us/sample - loss: 1.4815 - acc: 0.6870 - val_loss: 1.9876 - val_acc: 0.2961
Epoch 122/200
1000/1000 [==============================] - 0s 131us/sample - loss: 1.4768 - acc: 0.6870 - val_loss: 1.9868 - val_acc: 0.2963
Epoch 123/200
1000/1000 [==============================] - 0s 129us/sample - loss: 1.4722 - acc: 0.6900 - val_loss: 1.9860 - val_acc: 0.2969
Epoch 124/200
1000/1000 [==============================] - 0s 128us/sample - loss: 1.4676 - acc: 0.6930 - val_loss: 1.9852 - val_acc: 0.2967
Epoch 125/200
1000/1000 [==============================] - 0s 139us/sample - loss: 1.4630 - acc: 0.6930 - val_loss: 1.9845 - val_acc: 0.2968
Epoch 126/200

Epoch 178/200
1000/1000 [==============================] - 0s 147us/sample - loss: 1.2347 - acc: 0.7920 - val_loss: 1.9544 - val_acc: 0.3046
Epoch 179/200
1000/1000 [==============================] - 0s 138us/sample - loss: 1.2306 - acc: 0.7940 - val_loss: 1.9539 - val_acc: 0.3046
Epoch 180/200
1000/1000 [==============================] - 0s 144us/sample - loss: 1.2266 - acc: 0.7960 - val_loss: 1.9536 - val_acc: 0.3049
Epoch 181/200
1000/1000 [==============================] - 0s 145us/sample - loss: 1.2225 - acc: 0.7990 - val_loss: 1.9531 - val_acc: 0.3051
Epoch 182/200
1000/1000 [==============================] - 0s 141us/sample - loss: 1.2185 - acc: 0.8020 - val_loss: 1.9528 - val_acc: 0.3047
Epoch 183/200
1000/1000 [==============================] - 0s 153us/sample - loss: 1.2145 - acc: 0.8020 - val_loss: 1.9523 - val_acc: 0.3047
Epoch 184/200
1000/1000 [==============================] - 0s 162us/sample - loss: 1.2105 - acc: 0.8030 - val_loss: 1.9521 - val_acc: 0.3051
Epoch 185/200

In [8]:
# test data evaluation
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy: {:.4f}".format(test_accuracy))

10000/10000 [==============================] - 1s 51us/sample - loss: 1.9465 - acc: 0.3050
Test accuracy: 0.3050


In [9]:
# plot_fn(['CIFAR-10'], [hist.history['loss']], [hist.history['acc']], [hist.history['val_acc']])

In [10]:
model2 = create_model(optimizer='adam', drop_rate=0.5, batch_norm=True)
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              6293504   
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
activation_3 (Activation)    (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)             

In [11]:
# hist2 = model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))
hist2 = model2.fit_generator(data_flow, epochs=epochs, steps_per_epoch=math.ceil(aug_scale * len(x_train) / batch_size), validation_data=(x_test, y_test))

Epoch 1/200
5/5 [==============================] - 2s 456ms/step - loss: 2.7726 - acc: 0.2230 - val_loss: 6.2408 - val_acc: 0.1007
Epoch 2/200
5/5 [==============================] - 1s 291ms/step - loss: 2.2252 - acc: 0.3156 - val_loss: 6.2373 - val_acc: 0.1266
Epoch 3/200
5/5 [==============================] - 2s 306ms/step - loss: 2.0000 - acc: 0.3502 - val_loss: 7.4007 - val_acc: 0.1295
Epoch 4/200
5/5 [==============================] - 1s 297ms/step - loss: 1.8460 - acc: 0.3904 - val_loss: 6.9559 - val_acc: 0.1333
Epoch 5/200
5/5 [==============================] - 2s 337ms/step - loss: 1.7499 - acc: 0.4050 - val_loss: 7.1709 - val_acc: 0.1275
Epoch 6/200
5/5 [==============================] - 2s 306ms/step - loss: 1.6740 - acc: 0.4340 - val_loss: 6.7684 - val_acc: 0.1421
Epoch 7/200
5/5 [==============================] - 2s 305ms/step - loss: 1.5858 - acc: 0.4512 - val_loss: 5.4447 - val_acc: 0.1613
Epoch 8/200
5/5 [==============================] - 1s 294ms/step - loss: 1.5408 - a

5/5 [==============================] - 2s 304ms/step - loss: 0.3982 - acc: 0.8620 - val_loss: 2.8573 - val_acc: 0.3390
Epoch 64/200
5/5 [==============================] - 2s 318ms/step - loss: 0.3926 - acc: 0.8654 - val_loss: 2.6755 - val_acc: 0.3588
Epoch 65/200
5/5 [==============================] - 2s 303ms/step - loss: 0.3807 - acc: 0.8720 - val_loss: 2.7428 - val_acc: 0.3517
Epoch 66/200
5/5 [==============================] - 1s 299ms/step - loss: 0.3674 - acc: 0.8762 - val_loss: 2.7834 - val_acc: 0.3534
Epoch 67/200
5/5 [==============================] - 1s 298ms/step - loss: 0.3642 - acc: 0.8768 - val_loss: 2.7519 - val_acc: 0.3615
Epoch 68/200
5/5 [==============================] - 2s 301ms/step - loss: 0.3649 - acc: 0.8756 - val_loss: 2.8175 - val_acc: 0.3460
Epoch 69/200
5/5 [==============================] - 2s 303ms/step - loss: 0.3568 - acc: 0.8810 - val_loss: 2.8748 - val_acc: 0.3456
Epoch 70/200
5/5 [==============================] - 2s 302ms/step - loss: 0.3479 - acc: 0

KeyboardInterrupt: 

In [ ]:
# test data evaluation
test_loss2, test_accuracy2 = model2.evaluate(x_test, y_test)
print("Test accuracy: {:.4f}".format(test_accuracy2))

In [ ]:
train_loss_histories = [hist.history['loss'], hist2.history['loss']]
train_acc_histories = [hist.history['acc'], hist2.history['acc']]
test_acc_histories = [hist.history['val_acc'], hist2.history['val_acc']]

plot_fn(['model1','model2'], train_loss_histories, train_acc_histories, test_acc_histories)